In [1]:
from functools import cache
from itertools import product

#I have to admit this is heavily inspired from
#https://github.com/hyperneutrino/advent-of-code/blob/main/2024/revisited/day21p2.py#L26


#Learning. Recursive search, depth first, allows us to spend much less memory in teh search process.
#Problem was also heavily constrained by use of sequences. Im not sure Im up for stuff like this.

N={ 0+0j: '7', 1+0j: '8', 2+0j: '9',\
    0+1j: '4', 1+1j: '5', 2+1j: '6',\
    0+2j: '1', 1+2j: '2', 2+2j: '3',\
    0+3j: 'X', 1+3j: '0', 2+3j: 'A',}

#+---+---+---+
#| 7 | 8 | 9 |
#+---+---+---+
#| 4 | 5 | 6 |
#+---+---+---+
#| 1 | 2 | 3 |
#+---+---+---+
#    | 0 | A |
#    +---+---+


D={ 0+0j: 'X', 1+0j: '^', 2+0j: 'A',\
    0+1j: '<', 1+1j: 'v', 2+1j: '>'}

#    +---+---+
#    | ^ | A |
#+---+---+---+
#| < | v | > |
#+---+---+---+


dirs = { '>': 1, '^':-1j, '<':-1, 'v':1j}

P=["540A", "839A", "682A", "826A", "974A"]

def compute_seqs(keypad):
    is_numpad = 2j in keypad
    #print(is_numpad)
    for pa in keypad:
        if keypad[pa] == "X":
            ix = int(pa.real)
            iy = int(pa.imag)
        
    seqs = {}
    for pa in keypad:
        for pb in keypad:
            
            ax = int(pa.real)
            ay = int(pa.imag)
            
            bx = int(pb.real)
            by = int(pb.imag)
            
            a = keypad[pa]
            b = keypad[pb]
            
            if a == 'X' or b == 'X':
                continue
            dx = bx - ax
            dy = by - ay
            up_sideways_down = "^"*-dy + "<"*-dx + ">"*dx + "v"*dy + "A"
            down_sideways_up = "v"* dy + "<"*-dx + ">"*dx + "^"*-dy + "A"
            ap,sp = (up_sideways_down, down_sideways_up) if is_numpad else (down_sideways_up, up_sideways_down)
            
            seqs[(a,b)] = [ap]
            #convoluted way of dinding sequences            
            #I wpuld probably have just searched to keep it simpler
            if dx != 0 and dy != 0 and (ay != iy and ax != ix or by != iy and bx != ix):
                seqs[(a,b)].append(sp)
            
    return seqs

def solve(string, seqs):
    options = [seqs[(x,y)] for x,y in zip("A"+string,string)]
    return ["".join(x) for x in product(*options)]
        



def reference_compute_seqs(keypad):
    pos = []
    for r in range(len(keypad)):
        for c in range(len(keypad[r])):
            if keypad[r][c] is None: ir, ic = r, c
            else: pos.append((r, c))
    seqs = {}
    for rx, cx in pos:
        for ry, cy in pos:
            x = keypad[rx][cx]
            y = keypad[ry][cy]
            dr = ry - rx
            dc = cy - cx
            usd = "^" * -dr + "<" * -dc + ">" * dc + "v" * dr + "A"
            dsu = "v" * dr + "<" * -dc + ">" * dc + "^" * -dr + "A"
            ap, sp = (usd, dsu) if keypad == num_keypad else (dsu, usd)
            seqs[(x, y)] = [ap]
            if dr != 0 and dc != 0 and (rx != ir and cx != ic or ry != ir and cy != ic):
                seqs[(x, y)].append(sp)
    return seqs

num_seqs = compute_seqs(N)
dir_seqs = compute_seqs(D)

#
#dir_keypad = [
#    [None, "^", "A"],
#    ["<", "v", ">"]
#]
#num_keypad = [
#    ["7", "8", "9"],
#    ["4", "5", "6"],
#    ["1", "2", "3"],
#    [None, "0", "A"]
#]
#reference_dir_seqs = reference_compute_seqs(dir_keypad)
#reference_num_seqs = reference_compute_seqs(num_keypad)


#print("DIR")
#for k in set(list(dir_seqs.keys())+list(reference_dir_seqs.keys())):
#    if dir_seqs[k] != reference_dir_seqs[k]:
#        print(k, dir_seqs[k], reference_dir_seqs[k])

#print("NUM")
#for k in set(list(num_seqs.keys())+list(reference_num_seqs.keys())):
#    if num_seqs[k] != reference_num_seqs[k]:
#        print(k, num_seqs[k], reference_num_seqs[k])


dir_lengths = {key: len(value[0]) for key, value in dir_seqs.items()}        
#print(dir_lengths)

@cache
def compute_length(i):
    seq, depth = i
    if depth == 1:
        return sum(dir_lengths[(x,y)] for x,y in zip("A"+seq, seq))
    length = 0
    for x,y in zip("A"+seq, seq):
        length += min(compute_length((subseq, depth-1)) for subseq in dir_seqs[(x,y)])
    return length

def solution(P, depth):
    total = 0
    for line in P:
        inputs = [(i,depth) for i in solve(line, num_seqs)]
        length = min(map(compute_length, inputs))
        total += length * int(line[:-1])
    return total
    

print("part1:", solution(P,2), 278568)
print("part1:", solution(P,25), 341460772681012)



part1: 278568 278568
part1: 341460772681012 341460772681012


In [2]:
#this is solution from https://github.com/hyperneutrino/advent-of-code/blob/main/2024/revisited/day21p2.py#L26

#This was HAAAARD!!!


from collections import deque
from functools import cache
from itertools import product

def compute_seqs(keypad):
    pos = []
    for r in range(len(keypad)):
        for c in range(len(keypad[r])):
            if keypad[r][c] is None: ir, ic = r, c
            else: pos.append((r, c))
    seqs = {}
    for rx, cx in pos:
        for ry, cy in pos:
            x = keypad[rx][cx]
            y = keypad[ry][cy]
            dr = ry - rx
            dc = cy - cx
            usd = "^" * -dr + "<" * -dc + ">" * dc + "v" * dr + "A"
            dsu = "v" * dr + "<" * -dc + ">" * dc + "^" * -dr + "A"
            ap, sp = (usd, dsu) if keypad == num_keypad else (dsu, usd)
            seqs[(x, y)] = [ap]
            if dr != 0 and dc != 0 and (rx != ir and cx != ic or ry != ir and cy != ic):
                seqs[(x, y)].append(sp)
    return seqs

def solve(string, seqs):
    options = [seqs[(x, y)] for x, y in zip("A" + string, string)]
    return ["".join(x) for x in product(*options)]

num_keypad = [
    ["7", "8", "9"],
    ["4", "5", "6"],
    ["1", "2", "3"],
    [None, "0", "A"]
]

num_seqs = compute_seqs(num_keypad)

dir_keypad = [
    [None, "^", "A"],
    ["<", "v", ">"]
]

dir_seqs = compute_seqs(dir_keypad)
dir_lengths = {key: len(value[0]) for key, value in dir_seqs.items()}

@cache
def compute_length(seq, depth=25):
    if depth == 1:
        return sum(dir_lengths[(x, y)] for x, y in zip("A" + seq, seq))
    length = 0
    for x, y in zip("A" + seq, seq):
        length += min(compute_length(subseq, depth - 1) for subseq in dir_seqs[(x, y)])
    return length

total = 0

for line in P:
    inputs = solve(line, num_seqs)
    length = min(map(compute_length, inputs))
    total += length * int(line[:-1])

print(total)

341460772681012
